# Install Dependencies 

In [2]:
!pip install tensorflow==2.3.0
!pip install gym[all]
!pip install keras==2.6.0
!pip install keras-rl2

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\red_h\Envs\sanal_ortam2\Scripts\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\red_h\\AppData\\Local\\Temp\\pip-install-0rs6p_po\\box2d-py_7d7b3612e7854bc1be5862ea6bae5488\\setup.py'"'"'; __file__='"'"'C:\\Users\\red_h\\AppData\\Local\\Temp\\pip-install-0rs6p_po\\box2d-py_7d7b3612e7854bc1be5862ea6bae5488\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\red_h\AppData\Local\Temp\pip-wheel-w2nlp99p'
       cwd: C:\Users\red_h\AppData\Local\Temp\pip-install-0rs6p_po\box2d-py_7d7b3612e7854bc1be5862ea6bae5488\
  Complete output (16 lines):
  Using setuptools (version 57.4.0).
  running bdist_wheel
  running build
  running build_py
 


  Using cached lz4-3.1.10-cp38-cp38-win_amd64.whl
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached mujoco-py-1.50.1.68.tar.gz (120 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ale_py-0.7.3-cp38-cp38-win_amd64.whl (925 kB)
  Running setup.py clean for box2d-py
  Running setup.py clean for mujoco-py
Failed to build box2d-py mujoco-py
    Running setup.py install for mujoco-py: started
    Running setup.py install for mujoco-py: finished with status 'error'


# Test Random Environment with OPENAI GYM

In [1]:
import sys
sys.path.append("C:/Users/red_h/Envs/sanal_ortam2/Lib/site-packages")

In [2]:
import gym
import random

In [3]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n
print(states)
print(actions)

4
2


In [4]:
episodes = 10
for _ in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0, 1])
        observation, reward, done, info = env.step(action)
        score += reward
    print("Episode: {}  Score: {}  Done: {}".format(_, score, done))      

Episode: 1  Score: 15.0  Done: True
Episode: 2  Score: 16.0  Done: True
Episode: 3  Score: 14.0  Done: True
Episode: 4  Score: 25.0  Done: True
Episode: 5  Score: 19.0  Done: True
Episode: 6  Score: 20.0  Done: True
Episode: 7  Score: 12.0  Done: True
Episode: 8  Score: 28.0  Done: True
Episode: 9  Score: 19.0  Done: True
Episode: 10  Score: 10.0  Done: True


# Create a Deep Learning Model with Keras


In [5]:
import numpy as np

In [6]:
from tensorflow.keras.models import Sequential

In [7]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [32]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape= (1, states)))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

In [33]:
model =  build_model(states, actions)

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-Rl

In [35]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [36]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [38]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ["mae"])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1 )

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 12:29 - reward: 1.0000

C:\Users/red_h/Envs/sanal_ortam2/Lib/site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users/red_h/Envs/sanal_ortam2/Lib/site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
105 episodes - episode_reward: 93.990 [8.000, 200.000] - loss: 2.624 - mae: 19.047 - mean_q: 38.685

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: 1.0000
55 episodes - episode_reward: 180.600 [139.000, 200.000] - loss: 5.338 - mae: 37.518 - mean_q: 75.709

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: 1.0000
53 episodes - episode_reward: 192.094 [148.000, 200.000] - loss: 4.148 - mae: 38.853 - mean_q: 78.270

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: 1.0000
50 episodes - episode_reward: 198.900 [168.000, 200.000] - loss: 6.154 - mae: 39.132 - mean_q: 78.663

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 39s 4ms/step - reward: 1.0000
done, took 208.188 seconds


In [40]:
scores = dqn.test(env, nb_episodes = 100, visualize = False)
print(np.mean(scores.history["episode_reward"]))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [41]:
_ = dqn.test(env, nb_episodes = 50, visualize = True)

Testing for 50 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 2

# Reloading Agent from Memory

In [42]:
#deploy and save weights dqn
dqn.save_weights("dqn_weights.h5f", overwrite=True)

# Congratulations